<a href="https://colab.research.google.com/github/sonu2012/Information-Retrieval/blob/master/Information_Retrieval_SONU_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
pd.options.mode.chained_assignment = None
from google.colab import drive
drive.mount('/content/drive')
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/content/drive/MyDrive/NLP/archive'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        with open(os.path.join(dirname, filename)) as f:
            line_count = 0
            id_set = set()
            for l in f.readlines():
                line_count += 1
                if filename == "CISI.REL":
                    id_set.add(l.lstrip(" ").split(" ")[0])
                elif l.startswith(".I "):
                    id_set.add(l.split(" ")[1].strip())
            print(f"{filename} : {len(id_set)} items, over {line_count} lines.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/NLP/archive/CISI.ALL
CISI.ALL : 1460 items, over 108747 lines.
/content/drive/MyDrive/NLP/archive/CISI.QRY
CISI.QRY : 112 items, over 1525 lines.
/content/drive/MyDrive/NLP/archive/CISI.REL
CISI.REL : 76 items, over 3114 lines.
/content/drive/MyDrive/NLP/archive/featurematrix.csv
featurematrix.csv : 0 items, over 1461 lines.
/content/drive/MyDrive/NLP/archive/featurematrix (1).csv
featurematrix (1).csv : 0 items, over 1461 lines.
/content/drive/MyDrive/NLP/archive/featurematrix (4).csv
featurematrix (4).csv : 0 items, over 1461 lines.
/content/drive/MyDrive/NLP/archive/featurematrixlemm.csv
featurematrixlemm.csv : 0 items, over 1461 lines.
/content/drive/MyDrive/NLP/archive/dfdoc_mesh_tfidf.csv
dfdoc_mesh_tfidf.csv : 0 items, over 1461 lines.
/content/drive/MyDrive/NLP/archive/qrdoc_mesh_tfidf.csv
qrdoc_mesh_tfidf.csv : 0 items, over 11

UnicodeDecodeError: ignored

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
with open('/content/drive/MyDrive/NLP/archive/CISI.ALL') as f:
    lines = ""
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")
    
# print n lines
n = 10
for l in lines[:n]:
    print(l)

.I 1
.T 18 Editions of the Dewey Decimal Classifications
.A Comaromi, J.P.
.W The present study is a history of the DEWEY Decimal Classification.  The first edition of the DDC was published in 1876, the eighteenth edition in 1971, and future editions will continue to appear as needed.  In spite of the DDC's long and healthy life, however, its full story has never been told.  There have been biographies of Dewey that briefly describe his system, but this is the first attempt to provide a detailed history of the work that more than any other has spurred the growth of librarianship in this country and abroad.
.X 1	5	1 92	1	1 262	1	1 556	1	1 1004	1	1 1024	1	1 1024	1	1
.I 2
.T Use Made of Technical Libraries
.A Slater, M.
.W This report is an analysis of 6300 acts of use in 104 technical libraries in the United Kingdom. Library use is only one aspect of the wider pattern of information use.  Information transfer in libraries is restricted to the use of documents.  It takes no account of doc

In [6]:
doc_set = {}
doc_id = ""
doc_text = ""
for l in lines:
    if l.startswith(".I"):
        doc_id = l.split(" ")[1].strip()
    elif l.startswith(".X"):
        doc_set[doc_id] = doc_text.lstrip(" ")
        doc_id = ""
        doc_text = ""
    else:
        doc_text += l.strip()[3:] + " " # The first 3 characters of a line can be ignored.

# Print something to see the dictionary structure, etc.
print(f"Number of documents = {len(doc_set)}" + ".\n")
print(doc_set["5"]) # note that the dictionary indexes are strings, not numbers. 
        

Number of documents = 1460.

A Library Management Game: a report on a research project Brophy, P. Although the use of games in professional education has become widespread only during the last decade, the method has been used in a number of fields for many hundreds of years. Its origins have been traced to simple war games, used in military training when the "real thing" was either unavailable or too dangerous.  In more recent times, these games have become more and more sophisticated, and many now use large electronic computers to handle the complex calculations involved. Since 1956, when the first well-developed management game was introduced, the technique has spread rapidly into a wide variety of disciplines and today it is used at all levels of education, from primary school classes to courses for experienced professional men and women.  One of the main causes of this "game explosion" has been the rapid development of sophisticated management techniques, such as simulation and mat

In [7]:
with open('/content/drive/MyDrive/NLP/archive/CISI.QRY') as f:
    lines = ""
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")
    
qry_set = {}
qry_id = ""
for l in lines:
    if l.startswith(".I"):
        qry_id = l.split(" ")[1].strip()
    elif l.startswith(".W"):
        qry_set[qry_id] = l.strip()[3:]
        qry_id = ""
    
# Print something to see the dictionary structure, etc.
print(f"Number of queries = {len(qry_set)}" + ".\n")
print(qry_set["3"]) # note that the dictionary indexes are strings, not numbers.

Number of queries = 112.

What is information science?  Give definitions where possible.


In [8]:
rel_set = {}
with open('/content/drive/MyDrive/NLP/archive/CISI.REL') as f:
    for l in f.readlines():
        qry_id = l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[0]
        doc_id = l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[-1]
        if qry_id in rel_set:
            rel_set[qry_id].append(doc_id)
        else:
            rel_set[qry_id] = []
            rel_set[qry_id].append(doc_id)
        if qry_id == "7":
            print(l.strip("\n"))
    
# Print something to see the dictionary structure, etc.
print(f"\nNumber of mappings = {len(rel_set)}" + ".\n")
print(rel_set["7"]) # note that the dictionary indexes are strings, not numbers

     7    310	0	0.000000
     7    320	0	0.000000
     7    332	0	0.000000
     7    375	0	0.000000
     7    376	0	0.000000
     7    645	0	0.000000
     7    724	0	0.000000
     7    725	0	0.000000

Number of mappings = 76.

['310', '320', '332', '375', '376', '645', '724', '725']


In [9]:
docs = []
for i in range(1,len(doc_set)+1):
    docs.append(doc_set[str(i)])
    dfdoc=pd.DataFrame(docs,columns={'Docs'})

In [10]:
print(dfdoc)

                                                   Docs
0     18 Editions of the Dewey Decimal Classificatio...
1     Use Made of Technical Libraries Slater, M. Thi...
2     Two Kinds of Power An Essay on Bibliographic C...
3     Systems Analysis of a University Library; fina...
4     A Library Management Game: a report on a resea...
...                                                 ...
1455  World Dynamics Forrester, J.W. Over the last s...
1456  World Trends in Library Education Bramley, G. ...
1457  Legal Restrictions on Exploitation of the Pate...
1458  Language and Thought Poluskin, V.A. This book ...
1459  Modern Integral Information Systems for Chemis...

[1460 rows x 1 columns]


In [11]:
dfdoc["text_lower"] = dfdoc["Docs"].str.lower()
dfdoc.head()

,Docs,text_lower
0,18 Editions of the Dewey Decimal Classificatio...,18 editions of the dewey decimal classificatio...
1,"Use Made of Technical Libraries Slater, M. Thi...","use made of technical libraries slater, m. thi..."
2,Two Kinds of Power An Essay on Bibliographic C...,two kinds of power an essay on bibliographic c...
3,Systems Analysis of a University Library; fina...,systems analysis of a university library; fina...
4,A Library Management Game: a report on a resea...,a library management game: a report on a resea...


In [12]:
# drop the new column created in last cell
dfdoc.drop(["text_lower"], axis=1, inplace=True)

PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

dfdoc["text_wo_punct"] = dfdoc["Docs"].apply(lambda text: remove_punctuation(text))
dfdoc.head()

,Docs,text_wo_punct
0,18 Editions of the Dewey Decimal Classificatio...,18 Editions of the Dewey Decimal Classificatio...
1,"Use Made of Technical Libraries Slater, M. Thi...",Use Made of Technical Libraries Slater M This ...
2,Two Kinds of Power An Essay on Bibliographic C...,Two Kinds of Power An Essay on Bibliographic C...
3,Systems Analysis of a University Library; fina...,Systems Analysis of a University Library final...
4,A Library Management Game: a report on a resea...,A Library Management Game a report on a resear...


In [13]:
nltk.download('stopwords')
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [14]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

dfdoc["text_wo_stop"] = dfdoc["text_wo_punct"].apply(lambda text: remove_stopwords(text))
dfdoc.head()

,Docs,text_wo_punct,text_wo_stop
0,18 Editions of the Dewey Decimal Classificatio...,18 Editions of the Dewey Decimal Classificatio...,18 Editions Dewey Decimal Classifications Coma...
1,"Use Made of Technical Libraries Slater, M. Thi...",Use Made of Technical Libraries Slater M This ...,Use Made Technical Libraries Slater M This rep...
2,Two Kinds of Power An Essay on Bibliographic C...,Two Kinds of Power An Essay on Bibliographic C...,Two Kinds Power An Essay Bibliographic Control...
3,Systems Analysis of a University Library; fina...,Systems Analysis of a University Library final...,Systems Analysis University Library final repo...
4,A Library Management Game: a report on a resea...,A Library Management Game a report on a resear...,A Library Management Game report research proj...


In [15]:
dfdoc["text_to_lower"] = dfdoc["text_wo_stop"].str.lower()
dfdoc.head(10)

,Docs,text_wo_punct,text_wo_stop,text_to_lower
0,18 Editions of the Dewey Decimal Classificatio...,18 Editions of the Dewey Decimal Classificatio...,18 Editions Dewey Decimal Classifications Coma...,18 editions dewey decimal classifications coma...
1,"Use Made of Technical Libraries Slater, M. Thi...",Use Made of Technical Libraries Slater M This ...,Use Made Technical Libraries Slater M This rep...,use made technical libraries slater m this rep...
2,Two Kinds of Power An Essay on Bibliographic C...,Two Kinds of Power An Essay on Bibliographic C...,Two Kinds Power An Essay Bibliographic Control...,two kinds power an essay bibliographic control...
3,Systems Analysis of a University Library; fina...,Systems Analysis of a University Library final...,Systems Analysis University Library final repo...,systems analysis university library final repo...
4,A Library Management Game: a report on a resea...,A Library Management Game a report on a resear...,A Library Management Game report research proj...,a library management game report research proj...
5,"Abstracting Concepts and Methods Borko, H. Gra...",Abstracting Concepts and Methods Borko H Gradu...,Abstracting Concepts Methods Borko H Graduate ...,abstracting concepts methods borko h graduate ...
6,Academic Library Buildings A Guide to Architec...,Academic Library Buildings A Guide to Architec...,Academic Library Buildings A Guide Architectur...,academic library buildings a guide architectur...
7,The Academic Library Essays in Honor of Guy R....,The Academic Library Essays in Honor of Guy R ...,The Academic Library Essays Honor Guy R Lyle F...,the academic library essays honor guy r lyle f...
8,"Access to Libraries in College Hyman, R.T. Thi...",Access to Libraries in College Hyman RT This s...,Access Libraries College Hyman RT This study a...,access libraries college hyman rt this study a...
9,"Access to Periodical Resources Palmour, V.E. T...",Access to Periodical Resources Palmour VE The ...,Access Periodical Resources Palmour VE The pur...,access periodical resources palmour ve the pur...


In [16]:
from collections import Counter
cnt = Counter()
for text in dfdoc["text_to_lower"].values:
    for word in text.split():
        cnt[word] += 1
        
cnt.most_common(10)

[('the', 2090),
 ('information', 1573),
 ('library', 1207),
 ('system', 751),
 ('a', 727),
 ('libraries', 611),
 ('data', 586),
 ('research', 565),
 ('retrieval', 550),
 ('this', 534)]

In [ ]:
FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
def remove_freqwords(text):
    """custom function to remove the frequent words"""
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

dfdoc["text_wo_stopfreq"] = dfdoc["text_to_lower"].apply(lambda text: remove_freqwords(text))
dfdoc.head()

In [ ]:
# Drop the two columns which are no more needed 
dfdoc.drop(["text_wo_punct", "text_to_lower"], axis=1, inplace=True)

n_rare_words = 10
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
def remove_rarewords(text):
    """custom function to remove the rare words"""
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

dfdoc["text_wo_stopfreqrare"] = dfdoc["text_wo_stopfreq"].apply(lambda text: remove_rarewords(text))
dfdoc.head()

In [ ]:
from nltk.stem.porter import PorterStemmer

# Drop the two columns 
dfdoc.drop(["text_wo_stopfreq"], axis=1, inplace=True) 

stemmer = PorterStemmer()
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

dfdoc["text_stemmed"] = dfdoc["text_wo_stopfreqrare"].apply(lambda text: stem_words(text))
dfdoc.head()

In [ ]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

dfdoc["text_lemmatized"] = dfdoc["text_wo_stopfreqrare"].apply(lambda text: lemmatize_words(text))
dfdoc.head()

In [ ]:
dfdoc_mesh=pd.DataFrame(dfdoc["text_lemmatized"])

In [ ]:
dfdoc_mesh

In [ ]:
qrys = []
for i in range(1,len(qry_set)+1):
    qrys.append(qry_set[str(i)])
    qrdoc=pd.DataFrame(qrys,columns={'Query'})

In [ ]:
print(qrdoc)

In [ ]:
qrdoc["text_lower"] = qrdoc["Query"].str.lower()
qrdoc.head()

In [ ]:
qrdoc.drop(["text_lower"], axis=1, inplace=True)

PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

qrdoc["text_wo_punct"] = qrdoc["Query"].apply(lambda text: remove_punctuation(text))
qrdoc.head()

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

In [ ]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

qrdoc["text_wo_stop"] = qrdoc["text_wo_punct"].apply(lambda text: remove_stopwords(text))
qrdoc.head()

In [ ]:
qrdoc["text_to_lower"] = qrdoc["text_wo_stop"].str.lower()
qrdoc.head(10)

In [ ]:
from collections import Counter
cnt = Counter()
for text in qrdoc["text_to_lower"].values:
    for word in text.split():
        cnt[word] += 1
        
cnt.most_common(10)

In [ ]:
from nltk.stem.porter import PorterStemmer

# Drop the two columns 
#dfdoc.drop(["text_wo_stopfreq"], axis=1, inplace=True) 

stemmer = PorterStemmer()
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

qrdoc["text_stemmed"] = qrdoc["text_to_lower"].apply(lambda text: stem_words(text))
qrdoc.head()

In [ ]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

qrdoc["text_lemmatized"] = qrdoc["text_stemmed"].apply(lambda text: lemmatize_words(text))
qrdoc.head()

In [ ]:
qrdoc_mesh=pd.DataFrame(qrdoc["text_lemmatized"])

In [ ]:
qrdoc_mesh

In [ ]:
def splitbagorwords(text):
  document =text
  bagofwords=document.split(' ')
  return bagofwords

In [ ]:
qrdoc_mesh['bagG']=qrdoc_mesh['text_lemmatized'].apply(lambda row: splitbagorwords(row))

In [ ]:
qrdoc_mesh

In [ ]:
def uniq(text1):
  return set(text1)

In [ ]:
qrdoc_mesh['uniquewordscc']=qrdoc_mesh['bagG'].apply(lambda row: uniq(row))

In [ ]:
qrdoc_mesh

In [ ]:
qrdoc_mesh["UniwordCount"]=qrdoc_mesh['uniquewordscc'].apply(lambda x: len(str(x).split()))

In [ ]:
qrdoc_mesh

In [ ]:
def numwords(uniqtext,bagw):
  numOfWords = dict.fromkeys(uniqtext, 0)
  for word in bagw:
    numOfWords[word] += 1
  return numOfWords

In [ ]:
qrdoc_mesh["numwords"]=qrdoc_mesh.apply(lambda row:numwords(row['uniquewordscc'],row['bagG']),axis=1)

In [ ]:
qrdoc_mesh

In [ ]:
def computeTF(wordDict, bagOfWords):
    tfDict = {}
    bagOfWordsCount = len(bagOfWords)
    for word, count in wordDict.items():
        tfDict[word] = count / float(bagOfWordsCount)
    return tfDict

In [ ]:
qrdoc_mesh["tf"]=qrdoc_mesh.apply(lambda row:computeTF(row['numwords'],row['bagG']),axis=1)

In [ ]:
qrdoc_mesh.head(10)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv=CountVectorizer()
x=cv.fit_transform(dfdoc_mesh).toarray()

In [ ]:
x

In [ ]:
def splitbagorwords(text):
  document =text
  bagofwords=document.split(' ')
  return bagofwords

In [ ]:
dfdoc_mesh['bagG']=dfdoc_mesh['text_lemmatized'].apply(lambda row: splitbagorwords(row))

In [ ]:
dfdoc_mesh

In [ ]:
def uniq(text1):
  return set(text1)

In [ ]:
dfdoc_mesh['uniquewordscc']=dfdoc_mesh['bagG'].apply(lambda row: uniq(row))

In [ ]:
dfdoc_mesh

In [ ]:
dfdoc_mesh["UniwordCount"]=dfdoc_mesh['uniquewordscc'].apply(lambda x: len(str(x).split()))
dfdoc_mesh

In [ ]:
#datasethoc_mesh["CharCount"]=datasethoc_mesh['uniquewordscc'].apply(lambda x: len(x))

In [ ]:
def numwords(uniqtext,bagw):
  numOfWords = dict.fromkeys(uniqtext, 0)
  for word in bagw:
    numOfWords[word] += 1
  return numOfWords

In [ ]:
dfdoc_mesh["numwords"]=dfdoc_mesh.apply(lambda row:numwords(row['uniquewordscc'],row['bagG']),axis=1)

In [ ]:
dfdoc_mesh

In [ ]:
def computeTF(wordDict, bagOfWords):
    tfDict = {}
    bagOfWordsCount = len(bagOfWords)
    for word, count in wordDict.items():
        tfDict[word] = count / float(bagOfWordsCount)
    return tfDict

In [ ]:
dfdoc_mesh["tf"]=dfdoc_mesh.apply(lambda row:computeTF(row['numwords'],row['bagG']),axis=1)

In [ ]:
dfdoc_mesh.head(10)

In [ ]:
#datasethoc_mesh.to_csv("featurematrixlemm.csv")

In [ ]:
#from google.colab import files
#files.download('featurematrixlemm.csv')

In [ ]:
def computeIDF(documents):
    import math
    N = len(documents)
    
    idfDict = dict.fromkeys(documents[0].keys(), 0)
    for document in documents:
        for word, val in document.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log(N / float(val))
    return idfDict

In [ ]:
docs = dfdoc_mesh['text_lemmatized'].tolist()
tfidfv = TfidfVectorizer(analyzer='word', stop_words='english',)
tfidf_fit = tfidfv.fit(docs)

tfidf_doc = tfidf_fit.transform(docs)
tfidf_doc = pd.DataFrame.sparse.from_spmatrix(tfidf_doc)
tfidf_doc = tfidf_doc.apply(lambda num: num.astype(np.float64))
tfidf_doc.to_csv('/content/drive/MyDrive/NLP/archive/dfdoc_mesh_tfidf.csv')
#content/drive/MyDrive/NLP/archive/

In [ ]:
querys = qrdoc_mesh['text_lemmatized'].tolist()
tfidf_qry = tfidf_fit.transform(querys)
tfidf_qry = pd.DataFrame.sparse.from_spmatrix(tfidf_qry)
tfidf_qry = tfidf_qry.apply(lambda num: num.astype(np.float64))
tfidf_qry.to_csv('/content/drive/MyDrive/NLP/archive/qrdoc_mesh_tfidf.csv')

In [ ]:
tfidf_doc = pd.read_csv('/content/drive/MyDrive/NLP/archive/dfdoc_mesh_tfidf.csv')
tfidf_doc.drop(tfidf_doc.columns[0], axis = 1, inplace = True)
tfidf_quer = pd.read_csv('/content/drive/MyDrive/NLP/archive/qrdoc_mesh_tfidf.csv')
tfidf_quer.drop(tfidf_quer.columns[0], axis = 1, inplace = True)

In [ ]:
tfidf_doc.shape

In [ ]:
tfidf_quer.shape

In [ ]:
from scipy.spatial import distance

In [ ]:
def retrieveAll(tfidf_doc, tfidf_qry):
  docT = tfidf_doc.T
  qryT = tfidf_quer.T
  cosine_sim = []
  sorted_ind = []
  for i in qryT.columns:
    result = []
    vec1 = qryT[i]
    for j in docT.columns:
      vec2 = docT[j]
      result.append(1 - distance.cosine(vec1, vec2))
    sort = np.argsort(result)
    sort = sort[::-1]
    sorted_ind.append(sort)
    cosine_sim.append(result)
  return cosine_sim,sorted_ind

In [ ]:
cosine_sim, sorted_ind = retrieveAll(tfidf_doc, tfidf_quer)

cols = ['D'+str(i) for i in range(1,len(tfidf_doc)+1)]
cosine_mat = pd.DataFrame(cosine_sim, columns = cols)
cosine_mat.index = np.arange(1, len(cosine_mat) + 1)

sorted_index = pd.DataFrame(sorted_ind, columns = np.arange(1, len(tfidf_doc) + 1))
sorted_index.index = np.arange(1, len(sorted_index) + 1)

cosine_mat.to_csv('/content/drive/MyDrive/NLP/archive/doc_sw_cosine.csv')
sorted_index.to_csv('/content/drive/MyDrive/NLP/archive/doc_sw_sorted_index.csv')

In [ ]:
cos_matrix = pd.read_csv('/content/drive/MyDrive/NLP/archive/doc_sw_cosine.csv')
cos_matrix.drop(cos_matrix.columns[0], axis = 1, inplace = True)
sort_ind_value = pd.read_csv('/content/drive/MyDrive/NLP/archive/doc_sw_sorted_index.csv')
sort_ind_value.drop(sort_ind_value.columns[0], axis = 1, inplace = True)

cosine_sim_value = cos_matrix.values.tolist()
sorted_index_value = sort_ind_value.values.tolist()

In [ ]:
relation = []
for i in range(1,len(qry_set)+1):
  if i in rel_set.keys():
    relation.append(rel_set[i])
  else:
    relation.append(np.nan)

In [ ]:
def valid_ret(cos, sort_ind):
  valid = [int(ind) for ind in sort_ind if cos[ind] > 0]
  return valid

qrylist = ['Q'+str(i) for i in range(1,len(tfidf_quer)+1)] # if i in rel_set.keys()]
Result = pd.DataFrame(index = qrylist) 
Result['Cosine Distance Values'] = cosine_sim_value
Result['Sorted Doc Index'] = sorted_index_value 
#Result["Unique Doc Count"]=Result['Sorted Doc Index'].apply(lambda x: len(str(x).split()))
Result['Positive Retrieved'] = Result.apply(lambda row: valid_ret(row['Cosine Distance Values'], row['Sorted Doc Index']), axis = 1)


In [ ]:

Result.head(20)

In [ ]:
print(type((Result['Cosine Distance Values'].iloc[0])))
print(type((Result['Sorted Doc Index'].iloc[0])[0]))
print(type((Result['Positive Retrieved'].iloc[0])[0]))
print(type((Result['Cosine Distance Values'].iloc[0])[0]))

In [ ]:
def topMatch(sorted_index, rel_index, top):
    ret_index = sorted_index[0:top]
    rel_index = [l-1 for l in rel_index]
    rel_mtch = []
    for index in rel_index:
        if index in ret_index:
            rel_mtch.append(index)
    return rel_mtch

In [ ]:
def ret_rel(rel,ret):
  return (len(ret), len(rel))

# precision = relevant retrieved/total retrieved
def precision(ret_rel, n):
  ret, rel = ret_rel
  return ret/n

# recall =  relevent retrieved/total relevant
def recall(ret_rel,n):
  ret, rel = ret_rel
  return ret/rel

In [ ]:
total_documents = len(dfdoc_mesh["tf"])

In [ ]:
total_documents

In [ ]:
def inverse_doc_freq(word):
    try:
        word_occurance = dfdoc_mesh[word] + 1
    except:
        word_occurance = 1
    return np.log(total_documents/word_occurance)

In [ ]:
idfDict